<a href="https://colab.research.google.com/github/raj-aws/learnpython/blob/main/pymain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q google-genai wikipedia-api elevenlabs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00


In [4]:
import wikipediaapi
from google import genai
from elevenlabs.client import ElevenLabs
from elevenlabs import save
import os

# --- CONFIGURATION ---
# Replace these with your actual keys
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
ELEVENLABS_API_KEY = "YOUR_ELEVEN_LABS_KEY"

# Initialize Clients
client_gemini = genai.Client(api_key=GEMINI_API_KEY)
client_eleven = ElevenLabs(api_key=ELEVENLABS_API_KEY)

def get_wikipedia_content(topic):
    # Wikipedia-api requires a user-agent string
    wiki = wikipediaapi.Wikipedia(user_agent="HinglishProject/1.0", language='en')
    page = wiki.page(topic)
    if not page.exists():
        return f"Error: The page for {topic} does not exist."
    return page.summary[:4000]

def generate_hinglish_script(info):
    prompt = f"""
    CONTEXT DATA:
    {info}

    TASK:
    Write a natural, high-energy 2-minute conversation between two friends, ARJUN and KABIR.
    The topic is the information provided above.

    STYLE GUIDELINES:
    - Language: Hinglish (mix of Hindi and English).
    - Tone: Informal, like a casual WhatsApp voice note or a chai-break chat.
    - Conversational Fillers: Use 'umm', 'yaar', 'matlab', 'sahi hai', 'listen', 'ek minute'.
    - Human Elements: Include [laughter], [short pause], and [sigh] to avoid a robotic feel.
    - Structure: Arjun starts the conversation with a hook.

    OUTPUT FORMAT:
    Arjun: [dialogue]
    Kabir: [dialogue]
    """

    response = client_gemini.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt
    )
    return response.text

# --- RUNNING THE PIPELINE ---
topic = "Mumbai Indians"
print(f"Fetching data for {topic}...")
wiki_text = get_wikipedia_content(topic)

print("Generating Hinglish script...")
script = generate_hinglish_script(wiki_text)
print("-" * 30)
print(script)
print("-" * 30)

print("Synthesizing audio (ElevenLabs)...")
# Using the Multilingual v2 model is CRITICAL for the Hinglish accent
audio = client_eleven.generate(
    text=script,
    voice="George", # George has a warm, deep tone that handles bilingual text well
    model="eleven_multilingual_v2"
)

# Save to the Colab environment
output_filename = "podcast_sample.mp3"
save(audio, output_filename)
print(f"Success! Download your file '{output_filename}' from the left sidebar.")

Fetching data for Mumbai Indians...
Generating Hinglish script...


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}